In [1]:
import os

os.environ["GSUTIL_PATH"] = "/nas/ucb/abhay/google-cloud-sdk/bin/gsutil"
os.environ["CUDA_VISIBLE_DEVICES"] = "6"

from src.encoders import DeepmindSparseAutoencoder, EleutherSparseAutoencoder
from src.utils import *

#encoder = DeepmindSparseAutoencoder.load_gemma2_sae(0, 11 )

encoder = EleutherSparseAutoencoder.load_llama3_sae(24, v2=True)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
output = encoder.featurize_text(["Hello world", "My name is abhay"])
feature_indices, feature_activations = output['model.layers.24']

print(feature_indices.shape)
print(feature_activations.shape)

We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)


torch.Size([2, 10, 192])
torch.Size([2, 10, 192])


In [3]:
from src.database import FeatureDatabase
from datasets import load_dataset


database = FeatureDatabase(encoder)

if not os.path.exists("cached_sae_acts/"):
    # Load dataset
    dataset = load_dataset(
        "togethercomputer/RedPajama-Data-1T-Sample",
        split="train",
        # TODO: Maybe set this to False by default? But RPJ requires it.
        trust_remote_code=True,
    )
    # Cache SAE activations over dataset
    database.process_dataset(
        dataset=dataset,
        n_examples=100_000,
        folder_name="cached_sae_acts/",
        seed=42,
        example_seq_len=128,
        batch_size=64,
        text_column="text"
    )
else:
    database.load_from_disk("cached_sae_acts/")

In [4]:
ex = database.load_example(10)
print(ex)
print(len(ex.str_tokens))


<|begin_of_text|> all should burn for each—intensely, daily, and through word and work.”
“The last track, “Simulacrum“, closes this album. This song in so many ways cinches the thread tightly, both sonically and thematically, finishing this tapestry of ours. Contextually, it stands within the dualism and nihilism ever-present, the dance of life, and, the only enduring element of ourselves…our bone. That which will be stripped, bare, certain, and yet, hard enough persist. After all, this life is a dream…savor the seconds, the pain, and the glories
128


/nas/ucb/abhay/sae_experiments/src/database.py:117: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at ../torch/csrc/utils/tensor_numpy.cpp:206.)
  result[hook_name] = (torch.from_numpy(latent_indices), torch.from_numpy(latent_acts))


In [5]:
from src.helper_classes import Feature, Example

common_features = database.get_common_features(encoder.hook_name, k=1000)


100%|██████████| 98/98 [00:17<00:00,  5.61it/s]


In [16]:
import time
from circuitsvis.tokens import colored_tokens
from circuitsvis.activations import text_neuron_activations
import numpy as np

def get_examples(common_features, f_idx, n_top=50):
    feature = common_features[f_idx]
    max_activation_examples = feature.get_max_activating(n_top)

    tokens, acts = [], []
    for idx in range(n_top):
        tokens_idx, acts_idx = max_activation_examples[idx].get_tokens_feature_lists(feature)
        tokens.append(tokens_idx)
        acts.append(np.array([ [[x,],] for x in acts_idx]))
    
    text_neuron_activations(tokens, acts)
    return tokens, acts
    
tokens, acts = get_examples(common_features, 1)
text_neuron_activations(tokens, acts)